In [62]:
import pandas as pd
import numpy as np

In [63]:
#columns = ['current_state', 'action', 'next_state', 'probability']
#df = pd.DataFrame(pd.read_csv('mdp_1d_statedata.xls', header = None),columns=columns)
#df = pd.read_csv('mdp_1d_statedata.xls', header = None)
df = pd.read_csv('mdp_2d_data.xls')
#df = pd.read_excel('dataformdp.xlsm')
type(df)

pandas.core.frame.DataFrame

In [64]:
df.head()

,Inventory,Price,probability,nx_Inventory,nx_Price,action
0,10,5,0.6,9,5,a
1,10,5,0.4,8,5,a
2,10,5,0.3,9,5,a


In [65]:
#If the Data has one state feature or state is stored as list with all features
'''
columns = ['current_state', 'action', 'next_state', 'probability']
df.columns = columns
'''

"\ncolumns = ['current_state', 'action', 'next_state', 'probability']\ndf.columns = columns\n"

In [66]:
#If MDP has state features stored in multiple columns
'''
state_var = [col for col in df.columns if not col.startswith('nx') and col != 'action' and col != 'probability' and col != 'reward']
nx_state_var = ['nx_' + col  for col in state_var]
'''
nx_state_var = [col for col in df.columns if col.startswith('nx') and col != 'action' and col != 'probability' and col != 'reward']
state_var = [col.replace('nx_', '') for col in nx_state_var]

print(df.columns)
print(state_var)
print(nx_state_var)
'''
df['current_state'] = df[state_var].apply(lambda row: list(row), axis=1)
df['next_state'] = df[nx_state_var].apply(lambda row: list(row), axis=1)
'''

Index(['Inventory', 'Price', 'probability', 'nx_Inventory', 'nx_Price',
       'action'],
      dtype='object')
['Inventory', 'Price']
['nx_Inventory', 'nx_Price']


"\ndf['current_state'] = df[state_var].apply(lambda row: list(row), axis=1)\ndf['next_state'] = df[nx_state_var].apply(lambda row: list(row), axis=1)\n"

In [67]:
df.head()

,Inventory,Price,probability,nx_Inventory,nx_Price,action
0,10,5,0.6,9,5,a
1,10,5,0.4,8,5,a
2,10,5,0.3,9,5,a


In [68]:
#states = pd.concat([df[state_var], df[nx_state_var]], axis=0)
'''
nx_states = df[nx_state_var]
nx_states.columns = state_var
states = df[state_var].append(nx_states, ignore_index=True)
states = states.drop_duplicates()
'''
states = df[state_var].drop_duplicates()
#states = df[state_var].unique()
#states = df['current_state'].unique()
print(states)
print(type(states))

   Inventory  Price
0         10      5
<class 'pandas.core.frame.DataFrame'>


In [83]:
random_samples = []
num_samples = 100000  # Number of samples to generate

for _ in range(num_samples):
    #random_state = np.random.choice(states)
    random_state = states.sample(n=1) 
    state_value = random_state[state_var].values[0]
    #print(random_state)
    #print(random_state.columns == state_var)
    #print(random_state.columns)
    #print(type(random_state[state_var]))
    #print(state_value)
    #print(df[state_var] == state_value)
    #print(df[df[state_var] == state_value])
    random_state_tuple = pd.merge(df, random_state[state_var], on=state_var)
    #print(random_state_tuple)
    sample_action = (random_state_tuple.sample(n=1)).action.unique()[0]
    #print(sample_action)
    sample_row = random_state_tuple[random_state_tuple['action'] == sample_action].sample(n=1, weights='probability').iloc[0]

    # Extract the values from the selected row
    current_state = sample_row[state_var]
    action = sample_row['action']
    next_state = sample_row[nx_state_var]

    # Add the sample to the list
    random_samples.append(list(current_state) + [action] + list(next_state))
    #print(random_samples)

In [84]:
#df_samples = pd.DataFrame(random_samples, columns=['current_state', 'action', 'next_state'])
col = state_var + ['action'] + nx_state_var
print(col)
df_samples = pd.DataFrame(random_samples, columns=col)
df_samples.head()

['Inventory', 'Price', 'action', 'nx_Inventory', 'nx_Price']


,Inventory,Price,action,nx_Inventory,nx_Price
0,10,5,a,9,5
1,10,5,a,9,5
2,10,5,a,8,5
3,10,5,a,9,5
4,10,5,a,9,5


In [85]:
df_empirical_probs = df_samples.groupby(col).size().reset_index(name='count')
print(df_empirical_probs)
df_empirical_count = df_samples.groupby(state_var + ['action']).size().reset_index(name='count_total')
print(df_empirical_count)

df_empirical_probs = df_empirical_probs.merge(df_empirical_count, on=state_var + ['action'])
df_empirical_probs['empirical_probability'] = df_empirical_probs['count'] / df_empirical_probs['count_total']
print(df_empirical_probs)


   Inventory  Price action  nx_Inventory  nx_Price  count
0         10      5      a             8         5  30603
1         10      5      a             9         5  69397
   Inventory  Price action  count_total
0         10      5      a       100000
   Inventory  Price action  nx_Inventory  nx_Price  count  count_total  \
0         10      5      a             8         5  30603       100000   
1         10      5      a             9         5  69397       100000   

   empirical_probability  
0                0.30603  
1                0.69397  


In [87]:
df_merged = df.merge(df_empirical_probs, on=col, how='left')
# Check if the empirical probabilities match the specified probabilities
df_merged['prob_diff'] = df_merged['probability'] - df_merged['empirical_probability']
all_probs_match = np.allclose(df_merged['probability'], df_merged['empirical_probability'])

# Display the comparison results
print(df_merged)
print("\nDo all probabilities match?", all_probs_match)

   Inventory  Price  probability  nx_Inventory  nx_Price action  count  \
0         10      5          0.6             9         5      a  69397   
1         10      5          0.4             8         5      a  30603   
2         10      5          0.3             9         5      a  69397   

   count_total  empirical_probability  prob_diff  
0       100000                0.69397   -0.09397  
1       100000                0.30603    0.09397  
2       100000                0.69397   -0.39397  

Do all probabilities match? False
